In [44]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation as LDA
from scipy.stats import entropy

df = pd.read_csv("../sampled_hotel_1000.csv")
df


,Unnamed: 0,Hotel_Name,Review_Text,Posted_Date,Rating,Average_Rating,Num_of_Ratings,Helpfulness,is_photo,review_title,hotel_grade,employee_friendliness_score,facility_score,cleanliness_score,comfort_score,value_for_money_score,location_score,Crawled_date
0,0,tavistock,We have stayed here often and it is ideal Loca...,2024-06-12,9.0,7.7,5932,0,0,Will continue to stay here whenever we visit L...,3,8.7,7.6,8.2,8.0,7.5,9.1,2024-12-02
1,1,stgileshotel,"Its in a very convenient location, my mom whos...",2024-11-27,10.0,7.0,14989,0,0,Cozy and comfortable,3,8.1,6.9,7.3,7.3,7.0,9.4,2024-12-02
2,2,montanahotel,The staff were friendlyThe very small bathroom,2024-05-30,6.0,7.8,6248,0,0,We were on the 3rd floor were not told prior ...,3,9.0,7.7,8.2,8.2,8.0,9.4,2024-12-16
3,3,z-trafalgar,"Good location, was a 10 minute walk from the t...",2024-09-29,8.0,8.4,13923,0,0,Good functional hotel in great location,3,9.1,8.4,8.7,8.7,8.2,9.7,2024-12-02
4,4,blakemore,You hear a lot of noise from the other room do...,2023-11-10,7.0,8.3,12340,0,1,Great stay in a clean and tidy hotel,4,8.7,8.3,8.7,8.8,8.1,8.9,2024-12-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,995,crestfieldhotel,"The room was definitely clean, which is the mo...",2024-08-26,7.0,7.8,6515,0,0,"Noisy and small, but clean and friendly",2,8.6,7.6,8.4,8.1,7.9,9.4,2024-12-02
996,996,zedwell-trocaderor,modern look with nice finishroom very basic,2022-02-19,7.0,7.4,39497,0,0,great location which is what supports the pric...,0,8.1,7.2,7.9,7.9,7.5,9.6,2024-12-02
997,997,z-trafalgar,location was perfect,2022-07-05,9.0,8.4,13923,0,0,great stay and location,3,9.1,8.4,8.7,8.7,8.2,9.7,2024-12-02
998,998,radissonblugrafton,Big room with a desk and a table Good breakfas...,2023-06-28,8.0,7.9,9315,0,0,Nice rooms but very noisy over night,4,8.7,8.0,8.4,8.5,7.4,9.1,2024-12-02


# length

In [27]:
def count_words(column):
    return column.apply(lambda x: len(str(x).split()))

# 각 열의 단어 수 계산하여 새로운 열 추가
df['title_length'] = count_words(df['review_title'])
df['text_length'] = count_words(df['Review_Text'])

# 결과 출력
df

,Hotel_Name,Review_Text,Posted_Date,Rating,Average_Rating,Num_of_Ratings,Helpfulness,is_photo,review_title,hotel_grade,employee_friendliness_score,facility_score,cleanliness_score,comfort_score,value_for_money_score,location_score,Crawled_date,title_length,text_length
0,tavistock,We have stayed here often and it is ideal Loca...,2024-06-12,9.0,7.7,5932,0,0,Will continue to stay here whenever we visit L...,3,8.7,7.6,8.2,8.0,7.5,9.1,2024-12-02,9,31
1,stgileshotel,"Its in a very convenient location, my mom whos...",2024-11-27,10.0,7.0,14989,0,0,Cozy and comfortable,3,8.1,6.9,7.3,7.3,7.0,9.4,2024-12-02,3,47
2,montanahotel,The staff were friendlyThe very small bathroom,2024-05-30,6.0,7.8,6248,0,0,We were on the 3rd floor were not told prior ...,3,9.0,7.7,8.2,8.2,8.0,9.4,2024-12-16,24,7
3,z-trafalgar,"Good location, was a 10 minute walk from the t...",2024-09-29,8.0,8.4,13923,0,0,Good functional hotel in great location,3,9.1,8.4,8.7,8.7,8.2,9.7,2024-12-02,6,34
4,blakemore,You hear a lot of noise from the other room do...,2023-11-10,7.0,8.3,12340,0,1,Great stay in a clean and tidy hotel,4,8.7,8.3,8.7,8.8,8.1,8.9,2024-12-02,8,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,crestfieldhotel,"The room was definitely clean, which is the mo...",2024-08-26,7.0,7.8,6515,0,0,"Noisy and small, but clean and friendly",2,8.6,7.6,8.4,8.1,7.9,9.4,2024-12-02,7,100
996,zedwell-trocaderor,modern look with nice finishroom very basic,2022-02-19,7.0,7.4,39497,0,0,great location which is what supports the pric...,0,8.1,7.2,7.9,7.9,7.5,9.6,2024-12-02,18,7
997,z-trafalgar,location was perfect,2022-07-05,9.0,8.4,13923,0,0,great stay and location,3,9.1,8.4,8.7,8.7,8.2,9.7,2024-12-02,4,3
998,radissonblugrafton,Big room with a desk and a table Good breakfas...,2023-06-28,8.0,7.9,9315,0,0,Nice rooms but very noisy over night,4,8.7,8.0,8.4,8.5,7.4,9.1,2024-12-02,7,49


# time_lapesd

In [47]:
# Posted_Date와 Crawled_date를 datetime 형식으로 변환
df['Posted_Date'] = pd.to_datetime(df['Posted_Date'], format='%Y-%m-%d', errors='coerce')
df['Crawled_date'] = pd.to_datetime(df['Crawled_date'], format='%Y-%m-%d', errors='coerce')

# 날짜 차이 계산 (Crawled_date - Posted_Date)
df['time_lapsed'] = (df['Crawled_date'] - df['Posted_Date']).dt.days  # 일(day) 단위로 계산

df

,Unnamed: 0,Hotel_Name,Review_Text,Posted_Date,Rating,Average_Rating,Num_of_Ratings,Helpfulness,is_photo,review_title,hotel_grade,employee_friendliness_score,facility_score,cleanliness_score,comfort_score,value_for_money_score,location_score,Crawled_date,time_lapsed
0,0,tavistock,We have stayed here often and it is ideal Loca...,2024-06-12,9.0,7.7,5932,0,0,Will continue to stay here whenever we visit L...,3,8.7,7.6,8.2,8.0,7.5,9.1,2024-12-02,173
1,1,stgileshotel,"Its in a very convenient location, my mom whos...",2024-11-27,10.0,7.0,14989,0,0,Cozy and comfortable,3,8.1,6.9,7.3,7.3,7.0,9.4,2024-12-02,5
2,2,montanahotel,The staff were friendlyThe very small bathroom,2024-05-30,6.0,7.8,6248,0,0,We were on the 3rd floor were not told prior ...,3,9.0,7.7,8.2,8.2,8.0,9.4,2024-12-16,200
3,3,z-trafalgar,"Good location, was a 10 minute walk from the t...",2024-09-29,8.0,8.4,13923,0,0,Good functional hotel in great location,3,9.1,8.4,8.7,8.7,8.2,9.7,2024-12-02,64
4,4,blakemore,You hear a lot of noise from the other room do...,2023-11-10,7.0,8.3,12340,0,1,Great stay in a clean and tidy hotel,4,8.7,8.3,8.7,8.8,8.1,8.9,2024-12-02,388
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,995,crestfieldhotel,"The room was definitely clean, which is the mo...",2024-08-26,7.0,7.8,6515,0,0,"Noisy and small, but clean and friendly",2,8.6,7.6,8.4,8.1,7.9,9.4,2024-12-02,98
996,996,zedwell-trocaderor,modern look with nice finishroom very basic,2022-02-19,7.0,7.4,39497,0,0,great location which is what supports the pric...,0,8.1,7.2,7.9,7.9,7.5,9.6,2024-12-02,1017
997,997,z-trafalgar,location was perfect,2022-07-05,9.0,8.4,13923,0,0,great stay and location,3,9.1,8.4,8.7,8.7,8.2,9.7,2024-12-02,881
998,998,radissonblugrafton,Big room with a desk and a table Good breakfas...,2023-06-28,8.0,7.9,9315,0,0,Nice rooms but very noisy over night,4,8.7,8.0,8.4,8.5,7.4,9.1,2024-12-02,523


# star_dev

In [50]:
df['Deviation of star ratings'] = abs(df['Rating'] - df['Average_Rating'])
df

,Unnamed: 0,Hotel_Name,Review_Text,Posted_Date,Rating,Average_Rating,Num_of_Ratings,Helpfulness,is_photo,review_title,hotel_grade,employee_friendliness_score,facility_score,cleanliness_score,comfort_score,value_for_money_score,location_score,Crawled_date,time_lapsed,Deviation of star ratings
0,0,tavistock,We have stayed here often and it is ideal Loca...,2024-06-12,9.0,7.7,5932,0,0,Will continue to stay here whenever we visit L...,3,8.7,7.6,8.2,8.0,7.5,9.1,2024-12-02,173,1.3
1,1,stgileshotel,"Its in a very convenient location, my mom whos...",2024-11-27,10.0,7.0,14989,0,0,Cozy and comfortable,3,8.1,6.9,7.3,7.3,7.0,9.4,2024-12-02,5,3.0
2,2,montanahotel,The staff were friendlyThe very small bathroom,2024-05-30,6.0,7.8,6248,0,0,We were on the 3rd floor were not told prior ...,3,9.0,7.7,8.2,8.2,8.0,9.4,2024-12-16,200,1.8
3,3,z-trafalgar,"Good location, was a 10 minute walk from the t...",2024-09-29,8.0,8.4,13923,0,0,Good functional hotel in great location,3,9.1,8.4,8.7,8.7,8.2,9.7,2024-12-02,64,0.4
4,4,blakemore,You hear a lot of noise from the other room do...,2023-11-10,7.0,8.3,12340,0,1,Great stay in a clean and tidy hotel,4,8.7,8.3,8.7,8.8,8.1,8.9,2024-12-02,388,1.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,995,crestfieldhotel,"The room was definitely clean, which is the mo...",2024-08-26,7.0,7.8,6515,0,0,"Noisy and small, but clean and friendly",2,8.6,7.6,8.4,8.1,7.9,9.4,2024-12-02,98,0.8
996,996,zedwell-trocaderor,modern look with nice finishroom very basic,2022-02-19,7.0,7.4,39497,0,0,great location which is what supports the pric...,0,8.1,7.2,7.9,7.9,7.5,9.6,2024-12-02,1017,0.4
997,997,z-trafalgar,location was perfect,2022-07-05,9.0,8.4,13923,0,0,great stay and location,3,9.1,8.4,8.7,8.7,8.2,9.7,2024-12-02,881,0.6
998,998,radissonblugrafton,Big room with a desk and a table Good breakfas...,2023-06-28,8.0,7.9,9315,0,0,Nice rooms but very noisy over night,4,8.7,8.0,8.4,8.5,7.4,9.1,2024-12-02,523,0.1


# readability

In [57]:
import textstat


# 4. Readability 점수를 계산할 열 이름 설정
text_column = "Review_Text"  # CSV 파일의 텍스트가 저장된 열 이름

# 5. 새로운 열 추가 및 점수 계산
df["FOG Index"] = df[text_column].apply(lambda text: textstat.gunning_fog(str(text)))
df["Flesch Reading Ease"] = df[text_column].apply(lambda text: textstat.flesch_reading_ease(str(text)))

df

,Unnamed: 0,Hotel_Name,Review_Text,Posted_Date,Rating,Average_Rating,Num_of_Ratings,Helpfulness,is_photo,review_title,...,facility_score,cleanliness_score,comfort_score,value_for_money_score,location_score,Crawled_date,time_lapsed,Deviation of star ratings,FOG Index,Flesch Reading Ease
0,0,tavistock,We have stayed here often and it is ideal Loca...,2024-06-12,9.0,7.7,5932,0,0,Will continue to stay here whenever we visit L...,...,7.6,8.2,8.0,7.5,9.1,2024-12-02,173,1.3,13.69,65.39
1,1,stgileshotel,"Its in a very convenient location, my mom whos...",2024-11-27,10.0,7.0,14989,0,0,Cozy and comfortable,...,6.9,7.3,7.3,7.0,9.4,2024-12-02,5,3.0,22.20,40.69
2,2,montanahotel,The staff were friendlyThe very small bathroom,2024-05-30,6.0,7.8,6248,0,0,We were on the 3rd floor were not told prior ...,...,7.7,8.2,8.2,8.0,9.4,2024-12-16,200,1.8,2.80,89.75
3,3,z-trafalgar,"Good location, was a 10 minute walk from the t...",2024-09-29,8.0,8.4,13923,0,0,Good functional hotel in great location,...,8.4,8.7,8.7,8.2,9.7,2024-12-02,64,0.4,18.31,53.89
4,4,blakemore,You hear a lot of noise from the other room do...,2023-11-10,7.0,8.3,12340,0,1,Great stay in a clean and tidy hotel,...,8.3,8.7,8.8,8.1,8.9,2024-12-02,388,1.3,5.20,100.58
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,995,crestfieldhotel,"The room was definitely clean, which is the mo...",2024-08-26,7.0,7.8,6515,0,0,"Noisy and small, but clean and friendly",...,7.6,8.4,8.1,7.9,9.4,2024-12-02,98,0.8,41.60,3.82
996,996,zedwell-trocaderor,modern look with nice finishroom very basic,2022-02-19,7.0,7.4,39497,0,0,great location which is what supports the pric...,...,7.2,7.9,7.9,7.5,9.6,2024-12-02,1017,0.4,8.51,64.37
997,997,z-trafalgar,location was perfect,2022-07-05,9.0,8.4,13923,0,0,great stay and location,...,8.4,8.7,8.7,8.2,9.7,2024-12-02,881,0.6,14.53,34.59
998,998,radissonblugrafton,Big room with a desk and a table Good breakfas...,2023-06-28,8.0,7.9,9315,0,0,Nice rooms but very noisy over night,...,8.0,8.4,8.5,7.4,9.1,2024-12-02,523,0.1,20.42,55.58


# Depth

In [97]:
'''
from contextualized_topic_models.models.ctm import CombinedTopicModel
from contextualized_topic_models.utils.preprocessing import WhiteSpacePreprocessing
from contextualized_topic_models.utils.data_preparation import TopicModelDataPreparation
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
from scipy.stats import entropy

# Step 1: Preprocess reviews and prepare data for CTM
preprocessor = WhiteSpacePreprocessing(
    df['Review_Text'].tolist(), 
    stopwords_list=None  # You can pass a custom stopword list if needed
)
data_cleaned = preprocessor.preprocess()

# Prepare data for CTM
vectorizer = CountVectorizer(max_features=1000)
data_bow = vectorizer.fit_transform(data_cleaned)
data_vocab = vectorizer.get_feature_names_out()
data_ctm = TopicModelDataPreparation("paraphrase-MiniLM-L6-v2")
data_train = data_ctm.fit(data_cleaned, bow_matrix=data_bow)

# Step 2: Fit the CTM model
ctm_model = CombinedTopicModel(
    bow_size=len(data_vocab), 
    contextual_size=768,  # Dimension of embeddings (default for MiniLM)
    n_components=10,  # Number of topics
    num_epochs=20,  # Number of epochs
)
ctm_model.fit(data_train)

# Step 3: Compute topic proportions for each sentence
def get_sentence_topic_proportions(text, model, data_ctm):
    # Transform the sentence into embeddings and bow
    sentence_embeddings, sentence_bow = data_ctm.transform([text])
    topic_proportions = model.get_doc_topic_distribution(sentence_bow)
    return topic_proportions[0]  # Return the topic proportions

# Step 4: Aggregate sentence-level topic proportions to review-level topic proportions
def aggregate_to_review_level(review_text, model, data_ctm):
    sentences = review_text.split('.')  # Split review into sentences (basic sentence split)
    sentence_proportions = []

    for sentence in sentences:
        if sentence.strip():  # Skip empty sentences
            sentence_prop = get_sentence_topic_proportions(sentence.strip(), model, data_ctm)
            sentence_proportions.append(sentence_prop)

    # Calculate the average topic proportions for the entire review
    avg_topic_proportions = np.mean(sentence_proportions, axis=0)
    return avg_topic_proportions

# Step 5: Calculate Shannon entropy for each review
def calculate_entropy(topic_proportions):
    return entropy(topic_proportions, base=2)  # Shannon entropy

# Step 6: Compute ContentDepth for each review
def compute_content_depth(review_text, model, data_ctm):
    # Aggregate topic proportions to review-level
    avg_topic_proportions = aggregate_to_review_level(review_text, model, data_ctm)

    # Calculate entropy for the review
    entropy_value = calculate_entropy(avg_topic_proportions)

    # Normalize entropy by the number of sentences in the review
    num_sentences = len([s for s in review_text.split('.') if s.strip()])

    # Calculate ContentDepth as the negative value of the normalized entropy
    content_depth = -entropy_value / num_sentences if num_sentences > 0 else 0
    return content_depth

# Step 7: Apply ContentDepth calculation to each review in the DataFrame
df['ContentDepth'] = df['Review_Text'].apply(lambda x: compute_content_depth(x, ctm_model, data_ctm))

# Optional: Also calculate ContentBreadth (for example, inverse of ContentDepth, or other measures)
# df['ContentBreadth'] = -df['ContentDepth']  # Placeholder if you define breadth differently

# View the results
print(df[['Review_Text', 'ContentDepth']].head())
'''

ModuleNotFoundError: No module named 'contextualized_topic_models'

# Breadth

# 파일 저장

In [66]:
df.to_csv("../sampled_hotel_1000_calculated.csv")